In [1]:
!pip install scikit-learn
!pip install category_encoders
!pip install numpy
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from sklearn.model_selection import train_test_split
import category_encoders as ce
from category_encoders import BinaryEncoder

In [3]:
import pandas as pd
import numpy as np

In [4]:
df_evadidos = pd.read_csv('./outputs/dropout.csv', index_col=0)
df_formados = pd.read_csv('./outputs/graduated.csv', index_col=0)

In [5]:
df_formados

,Fluxo Escolar - Label,Campus (Nome),Matrícula retida?,Ingressantes em (Período Letivo),Modalidade do Curso,Nível de Ensino,Tipo de Modalidade (grupo),Cidade Aluno,Cód. da Matriz,Cód. do Curso,...,Tipo de Modalidade,Tipo_Escola_Origem,É matrícula retida neste período letivo?,Última Alteração de Situação,CH,CH. Disc. Obrigatórias,CH. Disc. Optativas,Duracao curso,Idade (Hoje),Periodos Cursados
indice,,,,,,,,,,,,,,,,,,,,,
3,Formados,Acaraú,Fora do prazo,20172,Subsequente,Técnico,Educação Profissional,Marco,7083,11204,...,Educação Profissional,E,Não Retida,2018-11-09,800,800,0,2,23,2
26,Formados,Acaraú,Dentro do prazo,20171,Subsequente,Técnico,Educação Profissional,Acaraú,7083,11204,...,Educação Profissional,E,Não Retida,2018-05-16,800,800,0,2,21,2
58,Formados,Acaraú,Dentro do prazo,20171,Subsequente,Técnico,Educação Profissional,Acaraú,7083,11204,...,Educação Profissional,E,Não Retida,2018-05-16,800,800,0,2,21,2
60,Formados,Acaraú,Dentro do prazo,20181,Subsequente,Técnico,Educação Profissional,Acaraú,5929,11203,...,Educação Profissional,E,Não Retida,2019-05-09,800,800,0,2,35,2
62,Formados,Acaraú,Dentro do prazo,20161,Subsequente,Técnico,Educação Profissional,Acaraú,5929,11203,...,Educação Profissional,E,Não Retida,2017-04-19,800,800,0,2,26,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3293,Formados,Acaraú,Fora do prazo,20151,Subsequente,Técnico,Educação Profissional,Itarema,5158,11200,...,Educação Profissional,E,Retida,2018-12-28,1240,1240,0,4,24,8
3305,Formados,Acaraú,Fora do prazo,20182,Fic,Básico,Regular,Acaraú,7274,111021,...,Regular,P,Retida,2019-10-18,160,160,0,2,40,2
3308,Formados,Acaraú,Fora do prazo,20172,Subsequente,Técnico,Educação Profissional,Acaraú,7083,11204,...,Educação Profissional,E,Não Retida,2018-11-09,800,800,0,2,31,2



### Functions and encoders to Normalize collumns

In [6]:
# A função para definir o intervalo da carga horária
def put_in_inverval(min: int, max: int, value: int) -> float:
    new_value = (value - min) / (max - min)
    return new_value

## Formatação dos dados

In [7]:
# Instância do encoder binário
encoder = ce.BinaryEncoder(cols=['Modalidade do Curso', 'Nível de Ensino', 'Tipo de Modalidade (grupo)', 'Tipo_Escola_Origem', 'Reserva vaga', 'Etnia', 'Estrutura Etária', 'Faixa Etária'])

# Atualmente só pode ser rodado uma vez tem que repensar a lógica;
df_juntos = [df_evadidos, df_formados]

# Para facilitar a implementação normalização dos dados nos datasets divididos ao mesmo tempo eu adicionei em um array
for df in df_juntos: 
    df['Fluxo Escolar - Label'] = df['Fluxo Escolar - Label'].map({
        'Formados': 1,
        'Evadidos': 0
    }).fillna(df['Fluxo Escolar - Label'])

    df['Matrícula retida?'] = df['Matrícula retida?'].map({
        'Dentro do prazo': 1,
        'Fora do prazo': 0
    }).fillna(df['Matrícula retida?'])

    df['É matrícula retida neste período letivo?'] = df['É matrícula retida neste período letivo?'].map({
        'Retida': 1,
        'Não Retida': 0
    }).fillna(df['É matrícula retida neste período letivo?']) 
    
    df['Ingressantes em (Período Letivo)'] = df['Ingressantes em (Período Letivo)'].map(lambda date: 0 if str(date)[-1] == '1' else 1)
    
    df['CH. Disc. Obrigatórias'] = df['CH. Disc. Obrigatórias'].map(lambda carga_horaria_obg: put_in_inverval(min=60, max=3260, value=carga_horaria_obg))
    
    df['Cidade Aluno'] = df['Cidade Aluno'].map(lambda cidade: 1 if cidade == 'Acaraú' else 0)
    
    df['Sexo'] =  df['Sexo'].map({'M': 1, 'F': 0})
    
# Colocar o algoritmo de binarização nos dataframes
df_evadidos = encoder.fit_transform(df_evadidos)
df_formados = encoder.fit_transform(df_formados)

In [8]:
df_evadidos

,Fluxo Escolar - Label,Campus (Nome),Matrícula retida?,Ingressantes em (Período Letivo),Modalidade do Curso_0,Modalidade do Curso_1,Modalidade do Curso_2,Nível de Ensino_0,Nível de Ensino_1,Nível de Ensino_2,...,Tipo_Escola_Origem_1,Tipo_Escola_Origem_2,É matrícula retida neste período letivo?,Última Alteração de Situação,CH,CH. Disc. Obrigatórias,CH. Disc. Optativas,Duracao curso,Idade (Hoje),Periodos Cursados
indice,,,,,,,,,,,,,,,,,,,,,
1,0,Acaraú,1,0,0,0,1,0,0,1,...,0,1,0,2016-02-26,1240,0.36875,0,4,28,2
4,0,Acaraú,1,1,0,1,0,0,1,0,...,1,0,0,2016-12-01,2940,0.87500,80,8,23,2
12,0,Acaraú,1,1,0,1,0,0,1,0,...,0,1,0,2018-12-27,2960,0.90625,0,8,35,7
21,0,Acaraú,1,0,0,1,1,0,1,1,...,0,1,0,2016-07-11,160,0.03125,0,2,33,1
23,0,Acaraú,1,1,0,1,1,0,1,1,...,0,1,0,2018-11-20,160,0.03125,0,2,20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3301,0,Acaraú,1,0,0,0,1,0,0,1,...,0,1,0,2015-12-29,1480,0.44375,0,4,23,1
3302,0,Acaraú,1,0,0,0,1,0,0,1,...,0,1,0,2019-12-04,1480,0.44375,0,4,23,2
3310,0,Acaraú,1,0,0,0,1,0,0,1,...,0,1,0,2016-06-06,1240,0.36875,0,4,28,2


In [9]:
df_formados

,Fluxo Escolar - Label,Campus (Nome),Matrícula retida?,Ingressantes em (Período Letivo),Modalidade do Curso_0,Modalidade do Curso_1,Modalidade do Curso_2,Nível de Ensino_0,Nível de Ensino_1,Nível de Ensino_2,...,Tipo_Escola_Origem_1,Tipo_Escola_Origem_2,É matrícula retida neste período letivo?,Última Alteração de Situação,CH,CH. Disc. Obrigatórias,CH. Disc. Optativas,Duracao curso,Idade (Hoje),Periodos Cursados
indice,,,,,,,,,,,,,,,,,,,,,
3,1,Acaraú,0,1,0,0,1,0,0,1,...,0,1,0,2018-11-09,800,0.23125,0,2,23,2
26,1,Acaraú,1,0,0,0,1,0,0,1,...,0,1,0,2018-05-16,800,0.23125,0,2,21,2
58,1,Acaraú,1,0,0,0,1,0,0,1,...,0,1,0,2018-05-16,800,0.23125,0,2,21,2
60,1,Acaraú,1,0,0,0,1,0,0,1,...,0,1,0,2019-05-09,800,0.23125,0,2,35,2
62,1,Acaraú,1,0,0,0,1,0,0,1,...,0,1,0,2017-04-19,800,0.23125,0,2,26,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3293,1,Acaraú,0,0,0,0,1,0,0,1,...,0,1,1,2018-12-28,1240,0.36875,0,4,24,8
3305,1,Acaraú,0,1,0,1,0,0,1,0,...,1,0,1,2019-10-18,160,0.03125,0,2,40,2
3308,1,Acaraú,0,1,0,0,1,0,0,1,...,0,1,0,2018-11-09,800,0.23125,0,2,31,2


### Dividing in tests

In [10]:
formados_x_treino, formados_x_teste, formados_y_treino, formados_y_teste = train_test_split(df_formados.drop('Fluxo Escolar - Label', axis=1), df_formados['Fluxo Escolar - Label'], test_size=0.2, random_state=2)


In [11]:
formados_x_treino

,Campus (Nome),Matrícula retida?,Ingressantes em (Período Letivo),Modalidade do Curso_0,Modalidade do Curso_1,Modalidade do Curso_2,Nível de Ensino_0,Nível de Ensino_1,Nível de Ensino_2,Tipo de Modalidade (grupo)_0,...,Tipo_Escola_Origem_1,Tipo_Escola_Origem_2,É matrícula retida neste período letivo?,Última Alteração de Situação,CH,CH. Disc. Obrigatórias,CH. Disc. Optativas,Duracao curso,Idade (Hoje),Periodos Cursados
indice,,,,,,,,,,,,,,,,,,,,,
130,Acaraú,0,0,0,0,1,0,0,1,0,...,0,1,1,2019-05-09,1480,0.44375,0,4,23,6
2096,Acaraú,1,0,0,1,0,0,1,0,1,...,0,1,0,2019-05-09,160,0.03125,0,2,20,2
3125,Acaraú,1,1,0,1,0,0,1,0,1,...,0,1,0,2016-05-27,160,0.03125,0,2,24,1
961,Acaraú,1,0,0,1,0,0,1,0,1,...,0,1,0,2016-10-18,160,0.03125,0,2,28,1
2317,Acaraú,0,0,0,0,1,0,0,1,0,...,0,1,0,2018-11-09,800,0.23125,0,2,24,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2917,Acaraú,0,1,0,0,1,0,0,1,0,...,0,1,0,2019-10-18,1080,0.31875,0,4,24,4
250,Acaraú,0,1,0,0,1,0,0,1,0,...,0,1,0,2019-10-18,800,0.23125,0,2,22,2
715,Acaraú,0,0,0,0,1,0,0,1,0,...,0,1,1,2017-11-01,800,0.23125,0,2,23,3


In [12]:
formados_y_treino

indice
130     1
2096    1
3125    1
961     1
2317    1
       ..
2917    1
250     1
715     1
185     1
1712    1
Name: Fluxo Escolar - Label, Length: 280, dtype: int64

In [13]:
evadidos_x_treino, evadidos_x_teste, evadidos_y_treino, evadidos_y_teste = train_test_split(df_evadidos.drop('Fluxo Escolar - Label', axis=1), df_evadidos['Fluxo Escolar - Label'], test_size=0.2, random_state=2)


In [14]:
evadidos_x_treino

,Campus (Nome),Matrícula retida?,Ingressantes em (Período Letivo),Modalidade do Curso_0,Modalidade do Curso_1,Modalidade do Curso_2,Nível de Ensino_0,Nível de Ensino_1,Nível de Ensino_2,Tipo de Modalidade (grupo)_0,...,Tipo_Escola_Origem_1,Tipo_Escola_Origem_2,É matrícula retida neste período letivo?,Última Alteração de Situação,CH,CH. Disc. Obrigatórias,CH. Disc. Optativas,Duracao curso,Idade (Hoje),Periodos Cursados
indice,,,,,,,,,,,,,,,,,,,,,
2703,Acaraú,0,1,0,0,1,0,0,1,0,...,0,1,1,2018-12-27,1080,0.31875,0,4,23,7
2495,Acaraú,0,1,0,1,0,0,1,0,1,...,0,1,1,2019-12-04,3200,0.98125,0,8,23,9
2261,Acaraú,1,1,0,1,0,0,1,0,1,...,0,1,0,2017-08-04,2960,0.90625,0,8,29,2
1780,Acaraú,1,0,0,0,1,0,0,1,0,...,0,1,0,2018-11-08,1240,0.36875,0,4,24,1
2239,Acaraú,1,1,0,0,1,0,0,1,0,...,0,1,0,2019-06-14,1240,0.36875,0,4,27,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,Acaraú,1,0,0,0,1,0,0,1,0,...,0,1,0,2018-12-27,800,0.23125,0,2,22,2
2070,Acaraú,1,1,0,0,1,0,0,1,0,...,0,1,0,2017-12-29,1480,0.44375,0,4,28,3
1727,Acaraú,0,1,0,0,1,0,0,1,0,...,0,1,1,2018-12-27,800,0.23125,0,2,25,3


In [15]:
evadidos_y_treino

indice
2703    0
2495    0
2261    0
1780    0
2239    0
       ..
1889    0
2070    0
1727    0
1861    0
623     0
Name: Fluxo Escolar - Label, Length: 760, dtype: int64

In [16]:
x_treino = pd.concat([formados_x_treino, evadidos_x_treino])

In [17]:
x_teste = pd.concat([formados_x_teste, evadidos_x_teste])

In [18]:
y_treino = pd.concat([formados_y_treino, evadidos_y_treino])

In [19]:
y_teste = pd.concat([formados_y_teste, evadidos_y_teste])

In [20]:
x_treino

,Campus (Nome),Matrícula retida?,Ingressantes em (Período Letivo),Modalidade do Curso_0,Modalidade do Curso_1,Modalidade do Curso_2,Nível de Ensino_0,Nível de Ensino_1,Nível de Ensino_2,Tipo de Modalidade (grupo)_0,...,Tipo_Escola_Origem_2,É matrícula retida neste período letivo?,Última Alteração de Situação,CH,CH. Disc. Obrigatórias,CH. Disc. Optativas,Duracao curso,Idade (Hoje),Periodos Cursados,Reserva vaga_3
indice,,,,,,,,,,,,,,,,,,,,,
130,Acaraú,0,0,0,0,1,0,0,1,0,...,1,1,2019-05-09,1480,0.44375,0,4,23,6,NaN
2096,Acaraú,1,0,0,1,0,0,1,0,1,...,1,0,2019-05-09,160,0.03125,0,2,20,2,NaN
3125,Acaraú,1,1,0,1,0,0,1,0,1,...,1,0,2016-05-27,160,0.03125,0,2,24,1,NaN
961,Acaraú,1,0,0,1,0,0,1,0,1,...,1,0,2016-10-18,160,0.03125,0,2,28,1,NaN
2317,Acaraú,0,0,0,0,1,0,0,1,0,...,1,0,2018-11-09,800,0.23125,0,2,24,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,Acaraú,1,0,0,0,1,0,0,1,0,...,1,0,2018-12-27,800,0.23125,0,2,22,2,1.0
2070,Acaraú,1,1,0,0,1,0,0,1,0,...,1,0,2017-12-29,1480,0.44375,0,4,28,3,0.0
1727,Acaraú,0,1,0,0,1,0,0,1,0,...,1,1,2018-12-27,800,0.23125,0,2,25,3,1.0


In [21]:
x_teste

,Campus (Nome),Matrícula retida?,Ingressantes em (Período Letivo),Modalidade do Curso_0,Modalidade do Curso_1,Modalidade do Curso_2,Nível de Ensino_0,Nível de Ensino_1,Nível de Ensino_2,Tipo de Modalidade (grupo)_0,...,Tipo_Escola_Origem_2,É matrícula retida neste período letivo?,Última Alteração de Situação,CH,CH. Disc. Obrigatórias,CH. Disc. Optativas,Duracao curso,Idade (Hoje),Periodos Cursados,Reserva vaga_3
indice,,,,,,,,,,,,,,,,,,,,,
257,Acaraú,0,1,0,1,0,0,1,0,1,...,1,0,2019-10-18,160,0.03125,0,2,24,2,NaN
2356,Acaraú,1,0,0,1,1,0,1,1,1,...,1,0,2019-06-05,3260,1.00000,0,8,25,8,NaN
2632,Acaraú,1,0,0,1,0,0,1,0,1,...,1,0,2016-10-18,160,0.03125,0,2,22,1,NaN
2620,Acaraú,1,1,0,1,0,0,1,0,1,...,1,0,2016-05-27,160,0.03125,0,2,31,1,NaN
2279,Acaraú,1,0,0,1,0,0,1,0,1,...,0,0,2016-10-18,160,0.03125,0,2,30,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,Acaraú,1,1,0,1,0,0,1,0,1,...,1,0,2019-06-06,2960,0.90625,0,8,24,7,1.0
3268,Acaraú,1,1,0,1,0,0,1,0,1,...,1,0,2016-06-06,2960,0.90625,0,8,25,1,1.0
81,Acaraú,0,1,0,1,1,0,1,1,1,...,1,1,2017-06-22,160,0.03125,0,2,28,2,1.0


In [22]:
y_treino

indice
130     1
2096    1
3125    1
961     1
2317    1
       ..
1889    0
2070    0
1727    0
1861    0
623     0
Name: Fluxo Escolar - Label, Length: 1040, dtype: int64

In [23]:
y_teste

indice
257     1
2356    1
2632    1
2620    1
2279    1
       ..
2496    0
3268    0
81      0
2156    0
687     0
Name: Fluxo Escolar - Label, Length: 260, dtype: int64

### Implementing Kneighbors Classifier

In [24]:
# Import classfier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier

# Seleção das colunas do x treino e teste que serão utilizadas no algoritmo

x_treino_dados_formatados = x_treino[[
    'Matrícula retida?', 'Ingressantes em (Período Letivo)', 
    'Modalidade do Curso_0', 'Modalidade do Curso_1', 'Modalidade do Curso_2', 
    'Nível de Ensino_0', 'Nível de Ensino_1', 'Nível de Ensino_2',
    'Tipo de Modalidade (grupo)_0', 'Tipo de Modalidade (grupo)_1',
    'CH. Disc. Obrigatórias', 'Cidade Aluno', 
    'Tipo_Escola_Origem_0', 'Tipo_Escola_Origem_1', 'Tipo_Escola_Origem_2', 
    'Reserva vaga_0', 'Reserva vaga_1', 'Reserva vaga_2',
    'Estrutura Etária_0', 'Estrutura Etária_1', 
    'Etnia_0', 'Etnia_1', 'Etnia_2', 
    'Faixa Etária_0', 'Faixa Etária_1', 'Faixa Etária_2', 'Faixa Etária_3',
    'Sexo'
]]

x_teste_dados_formatados =  x_teste[[
    'Matrícula retida?', 'Ingressantes em (Período Letivo)', 
    'Modalidade do Curso_0', 'Modalidade do Curso_1', 'Modalidade do Curso_2', 
    'Nível de Ensino_0', 'Nível de Ensino_1', 'Nível de Ensino_2',
    'Tipo de Modalidade (grupo)_0', 'Tipo de Modalidade (grupo)_1',
    'CH. Disc. Obrigatórias', 'Cidade Aluno', 
    'Tipo_Escola_Origem_0', 'Tipo_Escola_Origem_1', 'Tipo_Escola_Origem_2', 
    'Reserva vaga_0', 'Reserva vaga_1', 'Reserva vaga_2',
    'Estrutura Etária_0', 'Estrutura Etária_1', 
    'Etnia_0', 'Etnia_1', 'Etnia_2', 
    'Faixa Etária_0', 'Faixa Etária_1', 'Faixa Etária_2', 'Faixa Etária_3',
    'Sexo'
]]

# NeighborsClassifier and SGDClassifier instancias
clf = KNeighborsClassifier(n_neighbors=3, weights="distance")
sgd_class = SGDClassifier(alpha=0.001,loss='hinge',max_iter=100,tol=0.001,n_jobs=-1,early_stopping=True,n_iter_no_change=2)

clf.fit(
    x_treino_dados_formatados, y_treino
)

sgd_class.fit(x_treino_dados_formatados,y_treino)

predictions_clf = clf.predict(x_teste_dados_formatados)
predictions_sgd = sgd_class.predict(x_teste_dados_formatados)



In [25]:
# See the model accuracy
from sklearn.metrics import accuracy_score

score = accuracy_score(y_teste, predictions_clf)

print(f"Test Accuracy: {score}")
# Output - Test Accuracy : 0.97

Test Accuracy: 0.95


In [26]:
acc = accuracy_score(y_teste,predictions_sgd)
print(acc)

0.9307692307692308


In [36]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

matriz_confusao = confusion_matrix(y_teste, predictions_clf)

# print(matriz_confusao)

disp = ConfusionMatrixDisplay(confusion_matrix=matriz_confusao)
disp.plot(cmap=plt.cm.Blues)  # Usando um mapa de cores azul para o gráfico
plt.show()

ModuleNotFoundError: No module named 'matplotlib'